# Importing required package

In [1]:
import os
import re
import time 
import random
import numpy as np
import pandas as pd
from pprint import pprint

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn.datasets import load_files
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Loading Data

In [2]:
# list of categories for training model
categories = ['alt.atheism', 'talk.religion.misc', 'talk.politics.misc', 'comp.graphics', 'sci.space']
news_group_data = fetch_20newsgroups(subset='all', categories=categories, shuffle=True, remove=['headers', 'footers', 'quotes'], random_state=42)
# display
print("Data target_name:\n", news_group_data.target_names)
print("Data: ", len(news_group_data['data']))

Data target_name:
 ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.politics.misc', 'talk.religion.misc']
Data:  4162


In [3]:
# calculate frequency
target, count_target = np.unique(news_group_data.target, return_counts=True)
target_names = np.array(news_group_data.target_names)
print(list(zip(target_names, count_target)))

[('alt.atheism', 799), ('comp.graphics', 973), ('sci.space', 987), ('talk.politics.misc', 775), ('talk.religion.misc', 628)]


# Converting into pandas dataframe

In [4]:
complete_df = pd.DataFrame({'data':news_group_data.data, 'target':news_group_data.target})
complete_df.head()

,data,target
0,\nPeter Nelson posted a very eloquent response...,3
1,\nNo.\n\n\nNo. The library at Alexandria was p...,4
2,\nWouldn't this require a hyper-sphere. In 3-...,1
3,Forwarded from:\nPUBLIC INFORMATION OFFICE\nJE...,2
4,can someone tell me where i could find ansi or...,1


In [5]:
# size of data
len(complete_df)

4162

# Data Processing
    - WordNetLemmatizer() - removing all the special characters, spaces, character from start, and also converting into lower case
    - removing stopwords

In [6]:
# creating corpus list for iterating over text feature from data
corpus = list() 
# iterating over text data
for texts in range(0, len(complete_df)):
    # remove special character
    input_text = re.sub(r'\W', ' ', complete_df['data'][texts])
    # remove single character
    input_text = re.sub(r'\s+[a-zA-Z]\s+', ' ', input_text)
    # removing single character from start
    input_text = re.sub(r'\^[a-zA-Z]\s+', ' ', input_text)
    # multiple spaces into single spaces
    input_text = re.sub(r'\s+', ' ', input_text, flags=re.I)
    # removing prefixes
    input_text = re.sub(r'^b\s+', '', input_text)
    # into lower case
    input_text = input_text.lower()
    input_text = input_text.split()
    # initialise the WordNetLemmatizer()
    stemmer = WordNetLemmatizer()
    input_text = [stemmer.lemmatize(word) for word in input_text if not word in set(stopwords.words('english'))]
    input_text = ' '.join(input_text)
    # append list
    corpus.append(input_text)

# sanity check
print(corpus[:2])

['peter nelson posted eloquent response point talk politics misc need consume bandwidth meant clear intersection set liberal libertarian philosophy natural right government constitutional interpretation particular fit philosophy philosophy engages serious practical error endowing nine lawyer supreme court almost totalitarian authority completely outside consent consensus people supreme court nomination amazing political fist fight day control court rule country people court may well trying best job best benevolent oligarch even approve every supreme court decision ever eventually oligarch arise decimate hold dear try supreme court case jury problem would mitigated great deal would create broad non enumerated government power level european parlamentary democracy current de facto standard essentially engaged fundamental mistake except different body totalitarian virtually unchecked except plurality election death retirement government power like asking wind blow unless prove fact engagi

# Bag of Words Model
    - TF-IDF (Term Frequency - Inverse Document frequency) - Convert a collection of raw documents to a matrix of TF-IDF features 
    - CountVectoirzer - Convert a Collection of text documents to a matrix of token counts
    
Note: Chossing CountVectorizer() for this project

In [7]:
# define tokenizer function
def tokenizer(text):
    """
    Function to split the sentence into individual words.
    Args:
    Sentence: document containing Sentence (str)
    Returns:
    List of split individual words (Token)
    """
    return word_tokenize(text)

# sanity check
sentence = "Hi there, how have you been!?"
tokenizer(sentence)

['Hi', 'there', ',', 'how', 'have', 'you', 'been', '!', '?']

In [8]:
# # creating TF-IDF
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), tokenizer=tokenizer, max_features=1000000, use_idf=True)
# X = tfidf_vectorizer.fit_transform(corpus).toarray() # corpus is complete_df['data']
# y = complete_df.iloc[:, -1].values # labbels as target feature
# print(len(X[0])) # 

In [9]:
# Creating the Bag of Words model
count_vetorizer = CountVectorizer(stop_words='english', ngram_range=(1, 2), tokenizer=tokenizer, max_features=100000) # 10000
X = count_vetorizer.fit_transform(corpus).toarray() # corpus is complete_df['data']
y = complete_df.iloc[:, -1].values # labbels as target feature
print(len(X[0]))
print("Input feature's Shape: ", X.shape)

100000
Input feature's Shape:  (4162, 100000)


# Splitting the dataset into the Training and Test set
    - 80:20

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)
print("x_train: ", x_train.shape)
print("x_test: ", x_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)

x_train:  (3329, 100000)
x_test:  (833, 100000)
y_train:  (3329,)
y_test:  (833,)


# Define model
    - Naive Bayes - MultinomialNB()
    - SVM - Linear SVC
    - Neural Network - MLPClassifier()
    - comparing with Ensemble - Votting Classifier (hard votting)

In [11]:
from sklearn.ensemble import VotingClassifier
# define function - model_ensembler()
def model_ensembler():
    # model list
    models = list()
    # define models - SVM, Naive Bayes, NN
    models.append(('SVM', SVC(gamma='auto', kernel='linear')))
    models.append(('Naive Bayes', MultinomialNB()))
    models.append(('NN', MLPClassifier(alpha=1, max_iter=1000, hidden_layer_sizes=(2, 4, 6), solver='lbfgs', random_state=32)))
    # define votting classifier
    hard_voting_ensemble = VotingClassifier(estimators=models, voting='hard') # with largest sum of votes from models
    hard_voting_ensemble = hard_voting_ensemble.fit(x_train, y_train)
    
    return hard_voting_ensemble

# Evaluate model

In [ ]:
def evaluate_models():
    # empty dict
    models = dict()
    # define model for evaluation
    models['SVM'] = SVC(gamma='auto', kernel='linear')
    models['Naive Bayes'] = MultinomialNB()
    models['NN'] = MLPClassifier(alpha=1, max_iter=1000, hidden_layer_sizes=(2, 4, 6), solver='lbfgs', random_state=32) 
    models['Hard Voting Ensembler'] = model_ensembler()
    return models

# evaluation using Cross-validation - RepeatedStratifiedKFold()
def cross_validation_evaluation(model, x_train, y_train):
    cv = RepeatedStratifiedKFold(n_splits=5, random_state=1, n_repeats=3)
    scores=cross_val_score(model, x_train, y_train, cv=cv, n_jobs=-1, scoring='accuracy', error_score='raise')
    return scores

# get the models evaluation
models = evaluate_models()
# empty lists
results, model_names = list(), list()
for name, model in models.items():
    scores = cross_validation_evaluation(model, x_train, y_train)
    results.append(scores)
    model_names.append(name)
    print('>>>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


# Model comparison
    - Boxplot

In [ ]:
figure(num=None, figsize=(14, 8), dpi=80, facecolor='w', edgecolor='k')
plt.boxplot(results, labels=model_names)
plt.title("Classification model's performance comparison")
plt.show()

# Choosing best model

# Saving & loading model

In [ ]:
import pickle
# saving model
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# loading model
loading_model = pickle.load(open(filename, 'rb'))

# Evaluation metrics
    - Confusion metrix
    - Classification report

# Recocomender 
 - Future work (Recommender based on texts)
 - link: https://scikit-learn.org/stable/modules/metrics.html#cosine-similarity